# Sample pipeline for text feature extraction and evaluation

Adapted from http://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html

The dataset used in this example is the 20 newsgroups dataset which will be automatically downloaded and then cached and reused for the document classification example.

You can adjust the number of categories by giving their names to the dataset loader or setting them to None to get the 20 of them.

In [1]:
# Python Version Authors: Olivier Grisel <olivier.grisel@ensta.org>
#                         Peter Prettenhofer <peter.prettenhofer@gmail.com>
#                         Mathieu Blondel <mathieu@mblondel.org>
# License: BSD 3 clause

using ScikitLearn, Printf
using ScikitLearn.Pipelines: Pipeline
using ScikitLearn.GridSearch: GridSearchCV

@sk_import datasets: fetch_20newsgroups
@sk_import feature_extraction.text: (CountVectorizer, TfidfTransformer)
@sk_import linear_model: SGDClassifier

In [5]:
###############################################################################
# Load some categories from the training set
categories = [
    "alt.atheism",
    "talk.religion.misc",
]
# Uncomment the following to do the analysis on all the categories
#categories = None

print("Loading 20 newsgroups dataset for categories:")
print(categories)

data = fetch_20newsgroups(subset="train", categories=categories)
filenames = data["filenames"]
target_names = data["target_names"]
println("$(length(filenames)) documents")
println("$(length(target_names)) categories")
println()

cv = CountVectorizer()
fit_transform!(cv, data["data"][1:5], [0,0,1])

###############################################################################
# define a pipeline combining a text feature extractor with a simple
# classifier
pipeline = Pipeline([
    ("vect", CountVectorizer()),
    ("tfidf", TfidfTransformer()),
    ("clf", SGDClassifier()),
])

# uncommenting more parameters will give better exploring power but will
# increase processing time in a combinatorial way
parameters = Dict(
    "vect__max_df"=> (0.5, 0.75, 1.0),
    #'vect__max_features': (None, 5000, 10000, 50000),
    "vect__ngram_range"=> ((1, 1), (1, 2)),  # unigrams or bigrams
    #'tfidf__use_idf': (True, False),
    #'tfidf__norm': ('l1', 'l2'),
    "clf__alpha"=> (0.00001, 0.000001),
    "clf__penalty"=> ("l2", "elasticnet"),
    #'clf__n_iter': (10, 50, 80),
)

# find the best parameters for both the feature extraction and the
# classifier
grid_search = GridSearchCV(pipeline, parameters, n_jobs=1, verbose=1)

println("Performing grid search...")
println("pipeline:", [name for (name, _) in pipeline.steps])
println("parameters:")
println(parameters)
t0 = time()
N_samples = 100   # number of posts to train on
fit!(grid_search, data["data"][1:N_samples], data["target"][1:N_samples])
@printf("done in %0.3fs", time() - t0)
println()

@printf("Best score: %0.3f\n", grid_search.best_score_)
@printf("Best parameters set:\n")
best_parameters = get_params(grid_search.best_estimator_)
for param_name in keys(parameters)
    println("\t$param_name: $(best_parameters[param_name])")
end

Loading 20 newsgroups dataset for categories:ASCIIString["alt.atheism","talk.religion.misc"]857 documents
2 categories

Performing grid search...
pipeline:Any["vect","tfidf","clf"]
parameters:
Dict{ASCIIString,Any}("vect__max_df"=>(0.5,0.75,1.0),"clf__penalty"=>("l2","elasticnet"),"clf__alpha"=>(1.0e-5,1.0e-6),"vect__ngram_range"=>((1,1),(1,2)))
Fitting 3 folds for each of 24 candidates, totalling 72 fits
done in 7.997s
Best score: 0.860
Best parameters set:
	vect__max_df: 1.0
	clf__penalty: l2
	clf__alpha: 1.0e-5
	vect__ngram_range: (1,2)
